In [1]:
:opt no-lint

In [2]:
import Control.Monad.Trans.State

# 23 State
## 23.1 State
In Haskell state is data that exists in addition to the inputs and outputs of our functions, data
that can potentially change after each function is evaluated.

## 23.2 What is state?
The `State` type expresses a state that may change in the course of evaluating code without resort to mutability. The monadic interface for `State` is just a convenient way to handle it.
Handling state in that way:
1. Doesn’t require `IO`.
2. Is limited only to the data in our `State` container.
3. Maintains referential transparency.
4. Is explicit in the types of our functions.

## 23.3 Random numbers
A motivating example is given by the `next` function from `System.Random` which takes a state as input produces a tuple of a random value and the next state.
hence the n-th random number can be trivially generated by the following (pretty tedious) chaining:
```haskell
sg = mkStdGen 0
next (snd (next (snd (next sg))))
```
## 23.4 The State newtype
The definition is similar to the `Reader` one.
A newtype has the same underlying representation as the type it wraps. This is because the newtype wrapper disappears at compile time. Therefore, the function contained in the newtype must be isomorphic to the type it wraps. WWWWW riassumere

`State` is often used for random number generators, solvers, games, and carrying working memory while traversing a data structure.
Notice the compatibility of `random` signature with the `runState` one.
## 23.5 Throw down
In order to reduce the verbosity of the approach above, we can use the `state` function: a constructor that takes a `State`-like function and embeds it in the `State` monad transformer.
Example:

In [3]:
import System.Random

:t StateT
:t randomR

rollDie :: State StdGen Int
rollDie = state $ randomR (1, 6)

StateT :: forall s (m :: * -> *) a. (s -> m (a, s)) -> StateT s m a

randomR :: forall a g. (Random a, RandomGen g) => (a, a) -> g -> (a, g)

Now we can use monadic concatenations:

In [4]:
import Control.Applicative (liftA3)
import Control.Monad (replicateM)

rollDieThreeTimes :: State StdGen (Int, Int, Int)
rollDieThreeTimes = liftA3 (,,) rollDie rollDie rollDie

evalState rollDieThreeTimes (mkStdGen 42)

nDie :: Int -> State StdGen [Int]
nDie n = replicateM n rollDie

evalState (nDie 5) (mkStdGen 42)

(6,4,2)

[6,4,2,5,3]

Alternatively, `replicateM` can be replaced by a combination of `sequence` and `replicate`

In [5]:
nDie' :: Int -> State StdGen [Int]
nDie' n = sequence $ replicate n rollDie

evalState (nDie' 5) (mkStdGen 42)

[6,4,2,5,3]

The infinite list approach is not feasible here, as it implies computing the whole `[b]` in order to return a `m [b]`
### Exercises: Roll your own
1. Refactor `rollsToGetTwenty` so that the limit is an argument to the function:

In [6]:
rollsToGetN :: Int -> StdGen -> Int
rollsToGetN n = go 0 0
  where
    go :: Int -> Int -> StdGen -> Int
    go sum count gen
      | sum >= n = count
      | otherwise =
        let 
          (die, nextGen) = randomR (1, 6) gen
        in 
          go (sum + die) (count + 1) nextGen

2. Change `rollsToGetN` to record the series of dice that are rolled, in addition to the count of the total number of rolls:

In [7]:
rollsCountLogged :: Int -> StdGen -> (Int, [Int])
rollsCountLogged n = go 0 (0, [])
  where
    go :: Int -> (Int, [Int]) -> StdGen -> (Int, [Int])
    go sum (count, throws) gen
      | sum >= n = (count, throws)
      | otherwise =
        let 
          (die, nextGen) = randomR (1, 6) gen
        in 
          go (sum + die) (count + 1, die:throws) nextGen
          
rollsCountLogged 50 (mkStdGen 20)

(16,[4,2,4,4,5,4,1,4,2,3,2,4,1,1,3,6])

## 23.6 Write State for yourself
`State` type redefinition:

In [8]:
newtype Moi s a = Moi { runMoi :: s -> (a, s) }

### State Functor
Implement the `Functor` instance for `State`:

In [9]:
{-# LANGUAGE InstanceSigs #-}

instance Functor (Moi s) where
  fmap :: (a -> b) -> Moi s a -> Moi s b
  fmap f (Moi g) = Moi $ \s ->
    let
      (x, state) = g s
    in
      (f x, state)

f = (+1) <$> Moi (\s -> (0, s))
runMoi f 0

(1,0)

### State Applicative
Write the `Applicative` instance for `State`:

In [10]:
instance Applicative (Moi s) where
  pure :: a -> Moi s a
  pure a = Moi $ \s -> (a, s)
  (<*>) :: Moi s (a -> b) -> Moi s a -> Moi s b
  (Moi f) <*> (Moi g) = Moi $ \s ->
    let
      (fab, s')  = f s
      (x,   s'') = g s'
    in
      (fab x, s'')

This instance implies that the `<*>` operands cannot be computed in parallel.
### State Monad
Write the `Monad` instance for `State`:

In [11]:
instance Monad (Moi s) where
  return = pure
  (>>=) :: Moi s a -> (a -> Moi s b) -> Moi s b
  (Moi f) >>= g = Moi $ \s ->
    let
      (x, s') = f s
      (Moi f') = g x
    in
      f' s'

The `>>=` function runs the first state processor `f` on initial state `s0` to produce `s'` and `x`. Then it runs the second state processor `g x` on `s'` to produce the output state processor.

`Monad` is needed when the output dependes on previous states or values (e.g.: taking a random card from a deck, where card could have been removed by previous iteration). Otherwise, `Applicative` is enough (e.g.: throwing a dice several times)

## 23.7 Get a coding job with one weird trick
### FizzBuzz differently
Exercise: fix the reversing FizzBuzz without changing the underlying data structure:

In [12]:
fizzbuzzFromTo :: Integer -> Integer -> [String]
fizzbuzzFromTo from to = execState (mapM_ addResult (fromFromToToList to from)) []

fizzBuzz :: Integer -> String
fizzBuzz n | n `mod` 15 == 0 = "FizzBuzz"
  | n `mod` 5 == 0 = "Buzz"
  | n `mod` 3 == 0 = "Fizz"
  | otherwise = show n
  
addResult :: Integer -> State [String] ()
addResult n = do
  xs <- get
  let result = fizzBuzz n
  put (result : xs)

fromFromToToList :: (Num a, Enum a) => a -> a -> [a]
fromFromToToList from to = [from, from + signum (to - from)..to]

fizzbuzzFromTo 1 100

["1","2","Fizz","4","Buzz","Fizz","7","8","Fizz","Buzz","11","Fizz","13","14","FizzBuzz","16","17","Fizz","19","Buzz","Fizz","22","23","Fizz","Buzz","26","Fizz","28","29","FizzBuzz","31","32","Fizz","34","Buzz","Fizz","37","38","Fizz","Buzz","41","Fizz","43","44","FizzBuzz","46","47","Fizz","49","Buzz","Fizz","52","53","Fizz","Buzz","56","Fizz","58","59","FizzBuzz","61","62","Fizz","64","Buzz","Fizz","67","68","Fizz","Buzz","71","Fizz","73","74","FizzBuzz","76","77","Fizz","79","Buzz","Fizz","82","83","Fizz","Buzz","86","Fizz","88","89","FizzBuzz","91","92","Fizz","94","Buzz","Fizz","97","98","Fizz","Buzz"]

## 23.8 Chapter exercises
Write the following functions with the previously defined `Moi` type:

1. Construct a `Moi` where the state is also the value you return:

In [13]:
get :: Moi s s
get = Moi $ \x -> (x, x)

runMoi get "curryIsAmaze"

("curryIsAmaze","curryIsAmaze")

Essentially, `get` copies the state into the value

2. Construct a `Moi` where the resulting state is the argument provided, and the value defaults to unit:

In [14]:
put :: s -> Moi s ()
put s = Moi $ const ((), s)

runMoi (put "blah") "woot"

((),"blah")

Essentially, `put` discards the previous state

3. Run the `State` with `s` and get the state that results:

In [15]:
exec :: Moi s a -> s -> s
exec (Moi sa) = snd . sa

exec (put "wilma") "daphne"
exec get "scooby papu"

"wilma"

"scooby papu"

4. Run the `Moi` with `s` and get the value that results:

In [16]:
eval :: Moi s a -> s -> a
eval (Moi sa) = fst . sa

eval get "bunnicula"
eval get "stake a bunny"

"bunnicula"

"stake a bunny"

5. Write a function that applies a function to create a new `Moi`:

In [17]:
modify :: (s -> s) -> Moi s ()
modify ss = Moi $ \s -> ((), ss s)

f = modify (+1)
runMoi f 0
runMoi (f >> f >> f) 0

((),1)

((),3)

Pay attention to laziness. The stacked functions are not actually evaluated until `runState` is called (risk of memory leak)